In [1]:
import sionna.rt
from sionna.rt import load_scene, PlanarArray, Receiver, Transmitter

import mitsuba as mi
import drjit as dr
import numpy as np
import torch

In [2]:
# scene = load_scene(filename="cubes/cubes.xml")
scene = load_scene(filename="minis/row0_col0.xml")
scene.rx_array = PlanarArray(num_rows=1,
                          num_cols=1,
                          vertical_spacing=0.5,
                          horizontal_spacing=0.5,
                          polarization="V",
                          pattern="iso")

scene.tx_array = PlanarArray(num_rows=1,
                          num_cols=1,
                          vertical_spacing=0.5,
                          horizontal_spacing=0.5,
                          polarization="V",
                          pattern="iso")


In [3]:
# scene.preview()

In [4]:
mi_scene = scene.mi_scene

In [5]:
shapes = mi_scene.shapes_dr()

# Will throw an error if there is a non-mesh shape
shapes = mi.MeshPtr(shapes)

face_normals = [
    s.face_normal(dr.arange(dr.cuda.UInt, s.face_count())).torch().T for s in shapes
]
face_normals = torch.vstack(face_normals)

# Find the average between the 3 vertices of a triangle mesh
# This will be the source for ray intersection testing
face_averages = []
for s in shapes:
    indices = dr.arange(dr.cuda.UInt, s.face_count())
    vertex_indices = dr.ravel(s.face_indices(indices), order='F')
    vertices = s.vertex_position(vertex_indices)
    vertices = vertices.torch().T.reshape(-1, 3, 3)
    face_averages.append(vertices)

face_averages = torch.vstack(face_averages)
face_averages = face_averages.sum(1) / 3

# print(face_averages.shape)
# print(face_averages)

In [6]:
i = 0
for face_avg in face_averages[3:6]:
    # print(face_avg)
    rx = Receiver(name=f"rx{i}", position=face_avg)
    scene.add(rx)
    i += 1

# rx = Receiver(position=face_averages)
# scene.add(rx)

In [7]:
offset_normals = face_normals + face_averages

i = 0
for face_n in offset_normals:
    # print(face_n)
    tx = Transmitter(name=f"tx{i}", position=face_n)
    scene.add(tx)
    i += 1
# tx = Transmitter(position=offset_normals)
# scene.add(tx)

scene.preview()

Renderer(camera=PerspectiveCamera(aspect=1.31, children=(DirectionalLight(intensity=0.25, position=(0.0, 0.0, …

In [8]:
face_averages = mi.Point3f(face_averages.T)
face_normals = mi.Vector3f(face_normals.T)
print(face_averages)
print(face_normals)


[[343.981, 158.277, 0.666667],
 [358.563, 150.842, 1.33333],
 [328.538, 165.942, 0.666667],
 .. 2400 skipped ..,
 [149.496, 129.499, 6.16667],
 [145.624, 111.726, 8],
 [144.698, 122.124, 8]]
[[0.454239, 0.89088, -1.38526e-07],
 [0.454239, 0.89088, -0],
 [0.25843, 0.96603, 5.57558e-09],
 .. 2400 skipped ..,
 [0.477788, 0.878475, -0],
 [-0, 0, 1],
 [0, 0, 1]]


In [9]:
# Generate rays to be shot from each surface in 180 cone
# Currently generates 360 then filters based on dot product

from sionna.rt.utils import fibonacci_lattice, spawn_ray_from_sources
ray_per_source = 1000

sources = face_averages + (face_normals * 0.001)
rays = spawn_ray_from_sources(fibonacci_lattice, ray_per_source, sources) 
print(rays)

norms_expanded = dr.repeat(face_normals, ray_per_source)
print(norms_expanded)
# shouldn't this be positive dot prod???
active = dr.dot(rays.d, norms_expanded) > 0
print(active)

Ray[
  o=[[343.982, 158.278, 0.666667],
     [343.982, 158.278, 0.666667],
     [343.982, 158.278, 0.666667],
     [343.982, 158.278, 0.666667],
     [343.982, 158.278, 0.666667],
        2405990 skipped ..,
     [144.698, 122.124, 8.001],
     [144.698, 122.124, 8.001],
     [144.698, 122.124, 8.001],
     [144.698, 122.124, 8.001],
     [144.698, 122.124, 8.001]],
  d=[[0, 0, 1],
     [-0.0466353, -0.0427217, 0.997998],
     [0.00781566, 0.0890555, 0.995996],
     [0.0665843, -0.0868474, 0.993994],
     [-0.12437, 0.0219993, 0.991992],
        2405990 skipped ..,
     [0.118513, -0.0436636, -0.991992],
     [-0.101274, -0.0414674, -0.993994],
     [0.0381213, 0.0808625, -0.995996],
     [0.0187565, -0.0604002, -0.997998],
     [-0, 0, -1]],
  maxt=[3.40282e+38],
  time=[0, 0, 0, 0, 0,    2405990 skipped .., 0, 0, 0, 0, 0],
  wavelengths=[]
]
[[0.454239, 0.89088, -1.38526e-07],
 [0.454239, 0.89088, -1.38526e-07],
 [0.454239, 0.89088, -1.38526e-07],
 .. 2405994 skipped ..,
 [0, 0, 1],


In [10]:
interactions = mi_scene.ray_intersect_preliminary(rays, active=active)

count = 0
for s in mi_scene.shapes():
    count += s.primitive_count()

a=dr.select(interactions.is_valid() & (interactions.t > 1.5), interactions.prim_index, dr.cuda.UInt(count)).numpy()
b=dr.select(interactions.is_valid(), interactions.t, dr.cuda.Float32(-1)).numpy()

# for i in range(len(a)):
#     print(a[i], b[i])



# for i, b in enumerate(a):
#     print(b, interactions.prim_index[i])


# Need to check to make sure ray is intersecting with the correct side of plane!!!!!!
# Should just be checking to make sure [ray direction] dot [norm is positive] > threshold? 

In [11]:

sets = [set() for _ in range(count)]

for i in range(len(a)):
    vis = a[i]
    if vis < count:
        sets[i // ray_per_source].add(vis)

sets
for i, s in enumerate(face_averages.numpy().T):
    print(s, sets[i])

scene

[343.9812    158.27686     0.6666667] {512, 513, 514, 515, 516, 517, 518, 519, 6, 521, 522, 523, 524, 525, 526, 9, 3, 8, 11, 13, 532, 533, 534, 535, 4, 5, 27, 0, 10, 56, 57, 58, 59, 60, 61, 62, 63, 2, 12, 68, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 85, 86, 87, 94, 96, 97, 99, 100, 102, 103, 105, 106, 107, 113, 115, 125, 126, 128, 129, 130, 131, 134, 136, 142, 144, 145, 151, 153, 154, 155, 156, 158, 161, 164, 166, 168, 170, 183, 186, 490, 493, 494, 495, 496, 497, 498, 511}
[358.56348   150.84167     1.3333334] {0, 513, 514, 515, 516, 517, 518, 519, 2, 521, 522, 523, 524, 525, 526, 10, 12, 8, 13, 532, 533, 534, 535, 56, 57, 58, 59, 60, 61, 62, 63, 68, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 85, 86, 87, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 102, 103, 104, 105, 106, 107, 108, 112, 114, 115, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 135, 136, 137, 138, 143, 150, 152, 153, 154, 155, 156, 157, 158, 159, 161, 162, 163, 164, 165, 166, 168, 172, 174, 175, 184, 186,

interactions is 1d array num faces x rays per source long
reshape

In [ ]:
# This cell is lowkey pretty illegal but here's to being easier to implement
## THIS REQUIRES persist=True in call self._add_child(mesh, pmin, pmax, persist=True) in _plot_lines (preview.py)


starts = np.array([[0,0,0],
                   [1, 1, 1]])
ends = np.array([[5,5,5],
                 [-100, -100, 0]])
colors = np.array([1.0, 0.0, 0.0], dtype=np.float32)
colors = np.tile(colors, (starts.shape[0], 1))
print(colors)
print(colors.dtype)
width = np.array(1.0)
scene._preview_widget._plot_lines(starts, ends, colors, width)

In [12]:
scene.preview()

Renderer(camera=PerspectiveCamera(aspect=1.31, children=(DirectionalLight(intensity=0.25, matrixWorldNeedsUpda…